In [1]:
! pip install azure-storage-blob requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 28.5 MB/s eta 0:00:0000:01


In [1]:
import requests as req
import pandas as pd
from azure.storage.blob import BlobServiceClient
import os

In [ ]:
os.environ["AZURE_CONNECTION_STRING"] = "key"
os.environ["RAPID_API_KEY"] = "key"
os.environ["RAPID_API_HOST"] = "host"
os.environ["OPENROUTE_KEY"] = 'key'

In [6]:
connection_string = os.getenv('AZURE_CONNECTION_STRING')
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [7]:
def upload_blob(client, blob_name, local_file_path):
  container_name = "bdmcontainerp1"
  container_client = blob_service_client.get_container_client(container_name)

  with open(local_file_path, "rb") as data:
      container_client.upload_blob(name=blob_name, data=data, overwrite=True)

  blobs = [blob.name for blob in container_client.list_blobs()]
  if blob_name not in blobs:
    raise Exception("Blob was not uploaded")

# Weather Data
Check for more examples:  https://open-meteo.com/


In [8]:
latitude = 41.3874
longitude = 2.1686
start_date = '2025-03-11'
end_date = '2025-03-12'
hourly = 'temperature_2m,precipitation_probability,apparent_temperature,precipitation,cloud_cover,wind_speed_10m'
querystring = {
                'latitude':latitude,
                'longitude':longitude,
                'hourly':hourly,
                'start_date':start_date,
                'end_date':end_date
                }

endpoint = f'https://api.open-meteo.com/v1/forecast'
res = req.get(endpoint, params=querystring).json()
pd.DataFrame(res["hourly"], columns=res["hourly_units"].keys()).head()

,time,temperature_2m,precipitation_probability,apparent_temperature,precipitation,cloud_cover,wind_speed_10m
0,2025-03-11T00:00,10.5,0,9.5,0.0,88,4.4
1,2025-03-11T01:00,10.3,3,9.1,0.0,98,4.4
2,2025-03-11T02:00,10.0,3,8.7,0.0,94,4.7
3,2025-03-11T03:00,9.9,15,8.5,0.1,100,4.9
4,2025-03-11T04:00,10.2,35,9.0,0.1,100,3.7


# Accomodation

Check for more examples: https://rapidapi.com/DataCrawler/api/booking-com15/playground/apiendpoint_818c2744-8507-4071-829e-d080b667a06c

In [9]:
import requests

destination_ids = {"Barcelona": "-372490",
                   "Rome": "-126693",
                   "Madrid": "-390625",
                   "Paris": "-1456928"}

url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"

city = destination_ids["Barcelona"]
querystring = {
                "dest_id": city,
                "search_type":"CITY",
                "arrival_date":"2025-11-15",
                "departure_date":"2025-11-30",
                "adults":"2",
                "children_age":"0",
                "room_qty":"1",
                "page_number":"1",
                "units":"metric",
                "temperature_unit":"c",
                "languagecode":"en-us",
                "currency_code":"EUR"}

headers = {
	"x-rapidapi-key": os.environ["RAPID_API_KEY"],
	"x-rapidapi-host": os.environ["RAPID_API_HOST"]
}

res = req.get(url, headers=headers, params=querystring).json()

In [10]:
res

{'status': True,
 'message': 'Success',
 'timestamp': 1742669150641,
 'data': {'hotels': [{'hotel_id': 505434,
    'accessibilityLabel': 'C211 Barcelona Apartments.\n4 out of 5 for property rating.\n8.9 Excellent 1103 reviews.\n\u200eSarrià-St. Gervasi\u202c • \u200e2.1 km from downtown\u202c.\nMobile-only price.\n Entire apartment – 56\xa0m² : 2\xa0beds • 2\xa0bedrooms • 1\xa0living room • 2\xa0bathrooms.\nOriginal price 2457 EUR. Current price 2167 EUR..\n+146 EUR taxes and charges.',
    'property': {'blockIds': ['50543405_408978647_4_0_0'],
     'accuratePropertyClass': 0,
     'checkoutDate': '2025-11-30',
     'optOutFromGalleryChanges': 0,
     'id': 505434,
     'isFirstPage': True,
     'reviewScore': 8.9,
     'rankingPosition': 0,
     'checkinDate': '2025-11-15',
     'mainPhotoId': 526339672,
     'reviewScoreWord': 'Excellent',
     'isPreferred': True,
     'isPreferredPlus': True,
     'currency': 'EUR',
     'longitude': 2.14712575078011,
     'checkout': {'fromTime': 

# Transportation

Find out more in: https://openrouteservice.org/dev/#/api-docs/optimization/post
Alternatives: https://github.com/graphhopper/graphhopper/blob/master/README.md#Map-Matching
https://github.com/VROOM-Project/vroom/blob/master/docs/API.md